#### Setup for file

In [13]:
import pandas as pd
import numpy as np
import streamlit as st
import openai
import time
from bs4 import BeautifulSoup
import requests
import re
import os
import json

from openai._client import OpenAI

client = OpenAI(
    api_key=st.secrets["openai"]["api_key"],
)


In [14]:

today = time.strftime("%m_%d")
today_hour = time.strftime("%m_%d_%H:%M")

# Make a folder for today's data inside the data folder
if not os.path.exists("data/" + today):
    os.mkdir("data/" + today)


In [15]:

file_path_original_dataframe_csv = "data/tags_30_11.csv"
file_path_original_dataframe_excel = "data/tags_30_11.xlsx"

file_path_filter_dataframe_csv = "filter_tags.csv"
file_path_filter_dataframe_excel = "filter_tags.xlsx"

new_file_path_splitted_dataframe_excel = "data/"+today+"/splitted_tags_" + today_hour + ".xlsx"
new_file_path_splitted_dataframe_csv = "data/"+today+"/splitted_tags_" + today_hour + ".csv"

new_file_path_grouped_dataframe_excel = "data/"+today+"/grouped_tags_" + today_hour + ".xlsx"
new_file_path_grouped_dataframe_csv = "data/"+today+"/grouped_tags_" + today_hour + ".csv"

In [16]:
# load the dict from pickle file
import pickle
with open('./data/url_technical_text_dict.pkl', 'rb') as handle:
    url_text_dict = pickle.load(handle)

#### Explode the dataframe

In [17]:
df = pd.read_excel(file_path_original_dataframe_excel, usecols=["Product_Name","Product category", "url", "image_url"])
print(len(df))

# Split the 'Product category' column into multiple rows
df = df.assign(
    **{"Product category": df["Product category"].str.split(",")}
).explode("Product category")

# Trim whitespace from the 'Product category' column
df["Product category"] = df["Product category"].str.strip()


print(len(df))



182
264


#### Make new products_tags

In [18]:
# Create a new column to store the tags for each product
# df_tech_new_tags = pd.read_csv("./data/tags_30_11.csv", usecols=["Product_Name", "Product category", "url", "image_url"])
df_tech_new_tags = df
filter_dataframe = pd.read_csv(file_path_filter_dataframe_csv, usecols=["category", "product type", "technology", "application"])

# Set 'category' as the index of filter_dataframe for easier lookup
filter_dataframe.set_index('category', inplace=True)

# Iterate over the rows of the dataframe
for index, row in df_tech_new_tags.iterrows():
    print(index)
    product_category = str(row["Product category"])  # Convert to string
    # Get the tags for the product category
    if product_category in filter_dataframe.index:
        category_tags = filter_dataframe.loc[product_category].to_dict()  # Use a different variable
        print(row["Product_Name"])
        url = row["url"]
        website_text = url_text_dict.get(url, "")
        print(row["Product category"])
        print(category_tags)
        messages = [
            {
                "role": "system",
                "content": "You are a helpful assistant that generates relevant tags for products. Your responses should be a comma-separated list of tags. For example tag1, tag2, tag3, etc..."
            },
            {
                "role": "user",
                "content": f"For the product named '{row['Product_Name']}' in the category ' {row['Product category']} ' with the following description: '{website_text}', please select up to 2 tags from each top-level category from the provided list: {category_tags}. The top-level categories include 'Product Type', 'Technology', and 'Application'. The maximum total number of tags is 6, but you are not required to use all 6. Your response should be a comma-separated list of tags. Please note: avoid using 'tag' as a tag, and do not use the top-level category names as tags."
            }
        ]
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=messages
            # response_format={ "type": "json_object" }
        )
        tags = response.choices[0].message.content.strip()
        df_tech_new_tags.loc[index, "product_tags"] = tags
        # print(row["Product_Name"])
        print(tags)
        print("______________")
    else:
        print(f"Category '{product_category}' not found in filter_dataframe")

# save the dataframe as csv
df_tech_new_tags.to_csv(new_file_path_splitted_dataframe_csv, index=False)
df_tech_new_tags.to_excel(new_file_path_splitted_dataframe_excel, index=False)

0
AIS physical shore stations
Surveillance & monitoring
{'product type': 'Acoustic Control System, Autonomous Underwater Vehicle, Diver Detection Sonar, Leak Detection System', 'technology': '3D sonar technology, Acoustic Telemetry Link, AIS transponder, Autonomous Navigation', 'application': 'BOP operation, Marine Survey, Remote Sensing, Underwater Surveillance'}


/var/folders/3c/7nbqh9x54rsgdpqc1dpcxh7w0000gn/T/ipykernel_12876/1529623198.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AIS transponder, Autonomous Navigation, Underwater Surveillance, Remote Sensing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_tech_new_tags.loc[index, "product_tags"] = tags


AIS transponder, Autonomous Navigation, Underwater Surveillance, Remote Sensing
______________
1
AIS service management system
Surveillance & monitoring
{'product type': 'Acoustic Control System, Autonomous Underwater Vehicle, Diver Detection Sonar, Leak Detection System', 'technology': '3D sonar technology, Acoustic Telemetry Link, AIS transponder, Autonomous Navigation', 'application': 'BOP operation, Marine Survey, Remote Sensing, Underwater Surveillance'}
Acoustic Control System, AIS transponder, Marine Survey, Underwater Surveillance
______________
2
AIS space receivers
Communication solutions
{'product type': 'Modem, Radio, Subsea, Surface, Transceiver', 'technology': 'Acoustic Modem, Modem Softare, reconfigurable SDR, VDES technology', 'application': 'Maritime Communication, Navigation Systems, Subsea Communication, Underwater Mapping'}
Transceiver, Radio, reconfigurable SDR, VDES technology, Maritime Communication, Navigation Systems
______________
3
APOS - Operating station fo

#### Groupe the dataframe

In [19]:
import ast
def extract_tags(tags_str):
    tags_obj = ast.literal_eval(tags_str)
    if isinstance(tags_obj, dict):
        tags_values = tags_obj.values()
    elif isinstance(tags_obj, tuple):
        tags_values = tags_obj
    else:
        raise ValueError(f"Unexpected type {type(tags_obj)}: {tags_obj}")
    
    tags_list = []
    for value in tags_values:
        if isinstance(value, list):
            tags_list.append(', '.join(map(str.strip, value)))
        elif isinstance(value, dict):
            tags_list.append(str(value))
        else:
            tags_list.append(value.strip())
    return ', '.join(tags_list)

In [20]:
df_with_many_tags = pd.read_csv(new_file_path_splitted_dataframe_csv)
print(len(df_with_many_tags))
# df_with_many_tags['product_tags'] = df_with_many_tags['product_tags'].apply(extract_tags)
# combine all the tags with same product_name into one row. Let the name be the same as the first row, but for tags, combine them into two lists of tags. For the category, combine them into a list of categories. For website_url, image_url, choose the first one. 
df_with_many_tags = df_with_many_tags.groupby('Product_Name').agg({
    'product_tags': lambda x: ', '.join(set(map(lambda y: str(y).strip(), x))),
    'Product category': lambda x: ', '.join(map(lambda y: str(y).strip(), x)),
    'url': 'first',
    'image_url': 'first'
    
}).reset_index()
print(len(df_with_many_tags))

# save to excel
df_with_many_tags.to_excel(new_file_path_grouped_dataframe_excel, index=False)
df_with_many_tags.to_csv(new_file_path_grouped_dataframe_csv, index=False)

264
182


#### Merge with software tags

In [21]:
# if dataframe do not have column "is_software" then merge the column from software dataframe to the dataframe on the product name column
df_grouped_tags = pd.read_csv(new_file_path_grouped_dataframe_csv)
#check if the column is_software exists
if "is_software" not in df_grouped_tags.columns:
    # read the software dataframe
    df_software = pd.read_csv("data/tags_27_11.csv")
    # merge the column is_software to the dataframe on the product name column
    df_grouped_tags = pd.merge(df_grouped_tags, df_software[["Product_Name", "is_software"]], on="Product_Name", how="left")
    # save the dataframe
    print(df_grouped_tags.head())

                                        Product_Name  \
0                        AIS physical shore stations   
1                      AIS service management system   
2                                AIS space receivers   
3  APOS - Operating station for HiPAP and HPR Aco...   
4  APOS Survey - Surveyor's independent Operator ...   

                                        product_tags  \
0  AIS transponder, Autonomous Navigation, Underw...   
1  Acoustic Control System, AIS transponder, Mari...   
2  Transceiver, Radio, reconfigurable SDR, VDES t...   
3  Navigation system, Positioning system, Acousti...   
4  Navigation system, Positioning system, Acousti...   

                      Product category  \
0            Surveillance & monitoring   
1            Surveillance & monitoring   
2              Communication solutions   
3  Underwater navigation & positioning   
4  Underwater navigation & positioning   

                                                 url  \
0  https://www.ko